<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through NLTK, a Python NLP package.

**Part 1** (no actions required)

The dataset is composed of a set of sentences. Each sentence is a list of tuples of a word and a tag, as provided by human annotators.
You should split the data to train and test sets in the following way:


In [49]:
import numpy as np
import operator
import nltk
from nltk.corpus import treebank 
nltk.download('treebank')
print(f"Number of sentences: {len(treebank.tagged_sents())}")

train_data = treebank.tagged_sents()[:3000] 
test_data = treebank.tagged_sents()[3000:] 
print(train_data[0])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
Number of sentences: 3914
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


**Part 2**

Write a class simple_tagger, with methods *train* and *evaluate*. The method *train* receives the data as a list of treebank sentences, as presented above, and use it for training the tagger. In this case, it should learn a simple map of words to tags, defined as the most frequent tag for every word (in case there is more than one tag, select one randomly). The map should be stored as a class member for evaluation.

The method evaluate receives the data as a list of treebanl sentences, as presented above, and use it to evaluate the tagger performance. Specifically, it should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the map (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set. The function should return the two numbers: word level accuracy and sentence level accuracy.


In [0]:
from collections import Counter, defaultdict

class simple_tagger:
    def __init__(self):
        self.mft = None
        self.tags_dict = None

    def train(self, data):

        words = set()
        tags = Counter()

        # counter all pairs
        counter = Counter()
        for sen in data:
            words.update([pair[0] for pair in sen])
            tags.update([pair[1] for pair in sen])
            counter.update(Counter(sen))

        # most frequent tag/word
        self.mft = tags.most_common(1)[0]

        self.tags_dict = defaultdict(lambda: 0)
        for word in words:
            for tag in tags.keys():
                max_tag = self.tags_dict.get(word, None)
                if max_tag is None or (max_tag != tag and counter[(word, tag)] > counter[(word, max_tag)]):
                    self.tags_dict[word] = tag

    def evaluate(self, data):
        total_words = 0
        success_words = 0
        success_sentences = 0

        # iterate over all sentences
        for sen in data:
            # count the total number of words
            total_words += len(sen)

            # count match words in sentence
            success_sentence = sum([(self.tags_dict.get(word, self.mft) == tag) for word, tag in sen])

            # count match words
            success_words += success_sentence

            # count match sentences
            success_sentences += (success_sentence == len(sen))

        sentences_stats = success_sentences / len(data)
        words_stats = success_words / total_words
        return words_stats, sentences_stats

In [58]:
st = simple_tagger()
st.train(train_data)
print(st.tags_dict)
print(st.mft)

st.evaluate(test_data)

defaultdict(<function simple_tagger.train.<locals>.<lambda> at 0x7f00dcac68c8>, {'reflection': 'NN', 'favors': 'VBZ', 'declared': 'VBD', 'inventiveness': 'NN', 'fill': 'VB', 'fear': 'NN', 'Encouraged': 'JJ', '184': 'CD', 'introduces': 'VBZ', 'account': 'NN', 'minivans': 'NNS', 'Home': 'NNP', 'chemical': 'NN', 'disposables': 'NNS', 'barred': 'VBN', 'vetoes': 'NNS', 'biscuit': 'NN', 'point': 'NN', 'mobile': 'JJ', 'Braun': 'NNP', '#': '#', 'acceded': 'VBD', 'sorting': 'VBG', '*PPA*-2': '-NONE-', 'Can': 'MD', 'radio': 'NN', 'Burgundies': 'NNPS', 'Honolulu': 'NNP', 'aides': 'NNS', 'brokerage': 'NN', 'Basin': 'NNP', 'lottery': 'NN', 'cheerleaders': 'NNS', '*T*-1': '-NONE-', '*-137': '-NONE-', 'minimal': 'JJ', 'exercised': 'VBN', 'bargains': 'NNS', 'cornerstones': 'NNS', 'recruiting': 'VBG', 'landfill': 'NN', 'art': 'NN', 'top-yielding': 'JJ', 'overused': 'VBN', 'Inc': 'NNP', 'Nasty': 'JJ', 'planned': 'VBD', 'supply': 'NN', 'bang': 'NN', 'Her': 'PRP$', '*T*-243': '-NONE-', 'equally': 'RB', 'a

(0.8583207424994604, 0.06892778993435449)

**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence, using the Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. I implemented the Viterbi algorithm for you in the next block, so you can should either plug it into your code or write your own Viterbi version.

Additional guidance:
1. The matrix B represents the probabilities of seeing a word within each POS label.
Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way, every column in B represents the word that it’s id matches the index of the column.
2. During evaluation, you should first convert each word into it’s index and create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see OOV word, and decode every segment individually by Viterbi.


In [0]:
# Viterbi
def viterbi (word_list, A, B, Pi):

    # initialization
    T = len(word_list)
    N = A.shape[0] # number of tags

    delta_table = np.zeros((N, T)) # initialise delta table
    psi = np.zeros((N, T))  # initialise the best path table

    delta_table[:,0] = B[:, word_list[0]] * Pi

    for t in range(1, T):
        for s in range (0, N):
            trans_p = delta_table[:, t-1] * A[:, s]
            psi[s, t], delta_table[s, t] = max(enumerate(trans_p), key=operator.itemgetter(1))
            delta_table[s, t] = delta_table[s, t] * B[s, word_list[t]]

    # Back tracking
    seq = np.zeros(T);
    seq[T-1] =  delta_table[:, T-1].argmax()
    for t in range(T-1, 0, -1):
      #print(seq[t])
      seq[t-1] = psi[int(seq[t]),t]

    return seq

# A simple example to run the algorithm:

# A = np.array([[0.3, 0.7], [0.2, 0.8]])
# B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
# Pi = np.array([0.4, 0.6])
# print(viterbi([3, 3, 3, 3], A, B, Pi))

In [0]:
from collections import Counter, defaultdict
from random import randint

class hmm_tagger:

  def __init__(self):
    self.tags = None
    self.words = None
    self.A = None
    self.B = None
    self.Pi = None

  def train(self, data):

    # tmp - calculate words/tags dictionaries
    words_set = set()
    tags_set = set()
    # tmp - calculate self.B
    tags_counter = Counter()
    first_tags_counter = Counter()
    tags_prev_counter = defaultdict(lambda: Counter())
    tag_words_counter = defaultdict(lambda: Counter())
    for sen in data:
      prev_tag = None
      first_tags_counter[sen[0][1]] += 1
      for pair in sen:
        word = pair[0]
        tag = pair[1]        

        # dictionaries
        tags_set.add(tag)
        words_set.add(word)

        tags_counter[tag] += 1

        tag_words_counter[tag][word] += 1

        if prev_tag is not None:
          tags_prev_counter[tag][prev_tag] += 1
        prev_tag = tag

    # Calculate words/tags dictionaries
    self.tags = {k: i for i, k in enumerate(tags_set)}
    self.words = {k: i for i, k in enumerate(words_set)}

    # Calculate A
    self.A = np.zeros((len(self.tags), len(self.tags)))
    for tag, prev_tags in tags_prev_counter.items():
      tag_total = sum(prev_tags.values())
      for prev_tag in prev_tags:
        self.A[self.tags[tag]][self.tags[prev_tag]] = prev_tags[prev_tag] / tag_total

    # Calculate B
    self.B = np.zeros((len(self.tags), len(self.words)))
    for tag, words in tag_words_counter.items():
      tag_total = sum(words.values())
      for word in words:
        self.B[self.tags[tag]][self.words[word]] = words[word] / tag_total

    # Calculate Pi
    self.Pi = np.zeros(len(self.tags))
    total_tags = sum(first_tags_counter.values())
    for tag, count in first_tags_counter.items():
      self.Pi[self.tags[tag]] = count / total_tags
        

  def evaluate(self, data):
    total_words = 0
    success_words = 0
    sentences_stats = 0
    for sen in data:
      sen_len = len(sen)
      total_words += sen_len
      words_list = [self.words.get(pair[0], randint(0, len(self.words) - 1)) for pair in sen]
      tags_list = [self.tags.get(pair[1], randint(0, len(self.tags) - 1)) for pair in sen]

      results_list = viterbi(words_list, self.A, self.B, self.Pi)
      match_count = sum(t1 == t2 for t1, t2 in zip(tags_list, results_list))

      success_words += match_count
      sentences_stats += match_count / sen_len
    
    sentences_stats = sentences_stats / len(data)
    words_stats = success_words/total_words
    return words_stats, sentences_stats


In [0]:
hmm = hmm_tagger()
hmm.train(train_data)

In [64]:
print(hmm.tags)
print(hmm.words)
print(hmm.A)
print(hmm.B)
print(hmm.Pi)
print(np.sum(hmm.A[12]))
print(np.sum(hmm.B[22]))
print(np.sum(hmm.Pi))


{'VBP': 0, 'CC': 1, 'WP$': 2, 'FW': 3, 'JJR': 4, 'RBR': 5, ',': 6, 'TO': 7, 'WDT': 8, 'EX': 9, 'JJS': 10, '-LRB-': 11, 'NNS': 12, 'PRP$': 13, "''": 14, ':': 15, 'NNP': 16, 'CD': 17, '.': 18, 'IN': 19, 'RP': 20, 'NNPS': 21, '#': 22, 'JJ': 23, '-RRB-': 24, 'VBD': 25, 'WP': 26, '-NONE-': 27, '$': 28, 'VBN': 29, 'RBS': 30, 'DT': 31, 'PRP': 32, 'POS': 33, 'LS': 34, '``': 35, 'UH': 36, 'RB': 37, 'VB': 38, 'MD': 39, 'VBG': 40, 'PDT': 41, 'SYM': 42, 'NN': 43, 'VBZ': 44, 'WRB': 45}
{'reflection': 0, 'favors': 1, 'declared': 2, 'inventiveness': 3, 'fill': 4, 'fear': 5, 'Encouraged': 6, '184': 7, 'introduces': 8, 'account': 9, 'minivans': 10, 'Home': 11, 'chemical': 12, 'disposables': 13, 'barred': 14, 'vetoes': 15, 'biscuit': 16, 'point': 17, 'mobile': 18, 'Braun': 19, '#': 20, 'acceded': 21, 'sorting': 22, '*PPA*-2': 23, 'Can': 24, 'radio': 25, 'Burgundies': 26, 'Honolulu': 27, 'aides': 28, 'brokerage': 29, 'Basin': 30, 'lottery': 31, 'cheerleaders': 32, '*T*-1': 33, '*-137': 34, 'minimal': 35,

In [40]:
hmm.evaluate(test_data)

(0.5087416360889273, 0.5227569475043139)

**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK, over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [0]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)
print(tnt_pos_tagger.evaluate(test_data))

0.875545003237643


TODO: Print both, word level and sentence level accuracy for all the three taggers in a table.